# **Übung 7a** Programmierung mit Python mit Anwendungen aus dem Maschinellen Lernen



## Aufgabe 1
Die erste Aufgabe beschäftigt sich mit fortgeschrittenen Funktionen des Python Packages `pandas`.

In [ ]:
# Lädt die Datei sensor_data.csv in die Colab Umgebung
!gdown 1lzW5HFV8zS4htGDDJ6lM-v4CIfLJugj1

# Zum Anzeigen von Plots soll heute plotly verwendet werden
!pip install plotly

import plotly.express as px
import pandas as pd

# Aus Übung 7)
sensor_df = pd.read_csv('sensor_data.csv')
sensor_cleaned_df = sensor_df.dropna(
    subset=['AverageTemperatureFahr', 'City', 'Country']).reset_index(drop=True)

# Es werden dem bestehenden DataFrame zwei neue Spalten hinzugefügt.
# 'AverageTemperatureCelsius': Temperaturwerte in Celsius
sensor_cleaned_df['AverageTemperatureCelsius'] = (
    sensor_cleaned_df['AverageTemperatureFahr'] - 32) / 1.8
# 'datetime': Spalte mit Zeitstemplen für Plots
sensor_cleaned_df['datetime'] = pd.to_datetime(
    sensor_cleaned_df[['year', 'month', 'day']])

**Aufgabe 1.1 (pandas)** | Verwenden Sie `sensor_cleaned_df` und geben Sie zu einem Land (`Country`) aus:
- Wie viele Messwerte es gibt
- Aus wie vielen verschiedenen Städten (`City`) die Messwerte kommen
- Als konkatinierte Liste von Strings, um welche Städten (`City`) es sich handelt.

Sie können dazu die `groupby` und `agg` Funktionalität verwenden. Ausführliche Erläuterung: https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html


In [ ]:
# Group by Countries & Cities (to keep City information)
city_stats_df = sensor_cleaned_df[['City', 'Country', 'record_id']].groupby(
    ['City', 'Country']).agg({'record_id': 'count'}).rename(columns={'record_id': 'nr_measurements'}).reset_index()

# Group by Countries, aggregarte City information
city_stats_df = city_stats_df.groupby(
    ['Country']).agg({'City': ['count', ', '.join], 'nr_measurements': 'sum'}).rename(columns={'City': 'city_stats'}).reset_index()

city_stats_df.columns = ['Country',
                         'nr_cities', 'city_list', 'nr_measurements']

print(city_stats_df)

**Aufgabe 1.2 (pandas)** | Plotten Sie den Temperaturverlauf (`AverageTemperatureCelsius`) von Auckland über die Zeit (`datetime`). Verwenden Sie die `rolling` Methode, um eine zweite, geglättete Kurve darüber zulegen.


In [ ]:
single_city_df = sensor_cleaned_df[sensor_cleaned_df['City'] == 'Auckland']

px.line(
    x=single_city_df['datetime'],
    y=[single_city_df['AverageTemperatureCelsius'],single_city_df['AverageTemperatureCelsius'].rolling(window=240).mean()],
    color_discrete_sequence=["grey", "red"])


**Aufgabe 1.3 (pandas)** | Berechnen Sie mit Hilfe der Linearen Regression, die Sie bereits in Übung 4 kennen gelernt haben, den durchschnittlichen Temperaturanstieg in °C pro 100 Jahre für jede Stadt.
Sie können dazu die `groupby` und `apply` Funktionalität verwenden.

In [ ]:
from sklearn.linear_model import LinearRegression

def fitLinearRegression(df):
    fitted_regression = LinearRegression().fit(
        df['year'].values.reshape(-1, 1), df['AverageTemperatureCelsius'].values.reshape(-1, 1))
    ## Debug visualize
    # fig = px.line(
    #     x=df['year'],
    #     y=[df['AverageTemperatureCelsius'],fitted_regression.predict(df['year'].values.reshape(-1, 1)).flatten()],
    #     color_discrete_sequence=["grey", "red"])
    # fig.show()
    # _ = input()
    return fitted_regression.coef_[0, 0]


def aggregateTemperaturesPerMonth(df):
    return df.groupby(['month']).apply(fitLinearRegression).mean() * 100.


average_temp_increase = sensor_cleaned_df[['year', 'month', 'City', 'AverageTemperatureCelsius']].groupby(
    ['City']).apply(aggregateTemperaturesPerMonth)

print(average_temp_increase.reset_index().rename(
    columns={0: 'Delta°Cin100Years'}).sort_values('Delta°Cin100Years', ascending=False))